In [2]:
import pandas as pd
import numpy as np

# 1) Setup Parameters based on Roll Number
r = 102313063
ar = 0.05 * (r % 7)
br = 0.3 * ((r % 5) + 1)

print(f"Parameters for Roll Number {r}:")
print(f"ar = {ar}")
print(f"br = {br}")

# 2) Load and Prepare Data
file_path = 'data.csv'

try:
    df = pd.read_csv(
        file_path,
        encoding='latin1',
        engine='python',
        on_bad_lines='skip'
    )

    # Clean column names
    df.columns = df.columns.str.strip()

    # Find NO2 column (case-insensitive)
    col_match = [col for col in df.columns if col.lower() == 'no2']

    if not col_match:
        print("\nError: Could not find 'NO2' column.")
        print("Available columns are:", df.columns.tolist())
    else:
        target_col = col_match[0]
        print(f"\nFound target column: '{target_col}'")

        # Convert to numeric
        df[target_col] = pd.to_numeric(df[target_col], errors='coerce')

        # Drop NaNs
        data = df[[target_col]].dropna().copy()
        data.rename(columns={target_col: 'x'}, inplace=True)

        print(f"Data loaded successfully. Valid records: {len(data)}")

        # 3) Perform Transformation
        data['z'] = data['x'] + ar * np.sin(br * data['x'])

        print("\nTransformation Preview (first 5 rows):")
        print(data.head())

except FileNotFoundError:
    print("Error: 'data.csv' not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Parameters for Roll Number 102313063:
ar = 0.30000000000000004
br = 1.2

Found target column: 'no2'
Data loaded successfully. Valid records: 323938

Transformation Preview (first 5 rows):
      x          z
0  17.4  17.668863
1   7.0   7.256380
2  28.5  28.604985
3  14.7  14.419362
4   7.5   7.623636


In [3]:
#Learn Parameters (Method of Moments)

# 1. Calculate Mean (μ)
mu = data['z'].mean()

# 2. Calculate Variance (sigma squared) and Standard Deviation (sigma)
sigma_sq = data['z'].var()
sigma = data['z'].std()

# 3. Calculate Lambda (λ)
# Matching exponent -λ(z-μ)^2 to -1/(2σ^2) * (z-μ)^2
lam = 1 / (2 * sigma_sq)

# 4. Calculate c
# Standard Normal normalization constant
c = 1 / (sigma * np.sqrt(2 * np.pi))

print("-" * 30)
print("Step 2 Results: Learned Parameters")
print("-" * 30)
print(f"Mean (μ)      : {mu}")
print(f"Lambda (λ)    : {lam}")
print(f"Constant (c)  : {c}")

------------------------------
Step 2 Results: Learned Parameters
------------------------------
Mean (μ)      : 23.443847876006707
Lambda (λ)    : 0.0019986726441258962
Constant (c)  : 0.025222951093604614
